In [ ]:
# Installer les dépendances nécessaires
!pip install Pillow

import os
import shutil
import random
from PIL import Image, ImageDraw, ImageFilter
from math import radians, sin, cos

# Définir les sous-ensembles
SUBSETS = ['train', 'test', 'valid']

# Définir les chemins d'entrée et de sortie
BASE_INPUT_DIR = "/content"
BASE_OUTPUT_DIR = "/content/output"

# Créer la structure des dossiers de sortie
for subset in SUBSETS:
    images_input = os.path.join(BASE_INPUT_DIR, subset, "images")
    labels_input = os.path.join(BASE_INPUT_DIR, subset, "labels")

    images_output = os.path.join(BASE_OUTPUT_DIR, subset, "images")
    labels_output = os.path.join(BASE_OUTPUT_DIR, subset, "labels")

    os.makedirs(images_output, exist_ok=True)
    os.makedirs(labels_output, exist_ok=True)

# Fonctions de traitement des images (inchangées)
def save_image_with_conversion(img, path):
    _, ext = os.path.splitext(path)
    if ext.lower() in ['.jpg', '.jpeg']:
        img = img.convert("RGB")
    img.save(path)

def darken_left_side(image, darkness_factor=0.5):
    width, height = image.size
    overlay = Image.new('RGBA', (width, height), (0,0,0,0))
    draw = ImageDraw.Draw(overlay)
    for x in range(width):
        t = x / width
        alpha = int((1 - t) * darkness_factor * 120)
        draw.line((x, 0, x, height), fill=(0,0,0,alpha))
    return Image.alpha_composite(image.convert('RGBA'), overlay)

def create_rain_layer(width, height, num_drops, angle_variation, length_range, thickness_range, alpha_range, color_variation):
    rain_layer = Image.new('RGBA', (width, height), (0,0,0,0))
    draw = ImageDraw.Draw(rain_layer, 'RGBA')
    for _ in range(num_drops):
        x_start = random.randint(0, width)
        y_start = random.randint(0, height)
        length = random.randint(*length_range)
        thickness = random.randint(*thickness_range)
        angle = random.uniform(angle_variation[0], angle_variation[1])
        alpha = random.randint(*alpha_range)
        base_color = random.randint(color_variation[0], color_variation[1])
        color = (base_color, base_color, base_color, alpha)
        rad = radians(angle)
        x_end = x_start + int(sin(rad) * length)
        y_end = y_start + int(cos(rad) * length)
        draw.line((x_start, y_start, x_end, y_end), fill=color, width=thickness)
    rain_layer = rain_layer.filter(ImageFilter.GaussianBlur(radius=2))
    return rain_layer

def apply_rain_effect(image, intensity="heavy"):
    width, height = image.size
    img = image.convert("RGBA")
    img = darken_left_side(img, darkness_factor=0.5 if intensity == "heavy" else 0.3)

    if intensity == "heavy":
        base_num_drops = int((width * height) * 0.0009)
    else:
        base_num_drops = int((width * height) * 0.0005)

    layer = create_rain_layer(
        width, height,
        num_drops=base_num_drops,
        angle_variation=(-15, 15),
        length_range=(20, 60),
        thickness_range=(1,2),
        alpha_range=(80,180),
        color_variation=(230, 255)
    )

    combined = Image.alpha_composite(img, layer)
    return combined

def create_elliptical_snowflake(size, color, angle):
    flake_img = Image.new('RGBA', (size, size), (0,0,0,0))
    draw = ImageDraw.Draw(flake_img, 'RGBA')
    w = size
    h = max(1, size // 2)
    x0, y0 = (size - w)//2, (size - h)//2
    x1, y1 = x0+w, y0+h
    draw.ellipse((x0, y0, x1, y1), fill=color)
    return flake_img.rotate(angle, expand=1)

def create_star_snowflake(size, color, angle):
    flake_img = Image.new('RGBA', (size, size), (0,0,0,0))
    draw = ImageDraw.Draw(flake_img, 'RGBA')
    center = size//2
    draw.line((center-size//3, center-size//3, center+size//3, center+size//3), fill=color, width=1)
    draw.line((center+size//3, center-size//3, center-size//3, center+size//3), fill=color, width=1)
    draw.line((center-size//2, center, center+size//2, center), fill=color, width=1)
    return flake_img.rotate(angle, expand=1)

def create_polygon_snowflake(size, color, angle):
    flake_img = Image.new('RGBA', (size, size), (0,0,0,0))
    draw = ImageDraw.Draw(flake_img, 'RGBA')
    center = size//2
    num_points = random.randint(4,6)
    points = []
    for _ in range(num_points):
        r = random.randint(size//4, size//2)
        ang = random.uniform(0, 360)
        rad = radians(ang)
        px = center + int(r*sin(rad))
        py = center + int(r*cos(rad))
        points.append((px,py))
    draw.polygon(points, fill=color)
    return flake_img.rotate(angle, expand=1)

def create_snowflake(size, color, angle):
    flake_type = random.choice(["ellipse", "star", "polygon"])
    if flake_type == "ellipse":
        return create_elliptical_snowflake(size, color, angle)
    elif flake_type == "star":
        return create_star_snowflake(size, color, angle)
    else:
        return create_polygon_snowflake(size, color, angle)

def apply_snow_effect(image, intensity="heavy"):
    width, height = image.size
    img = image.convert("RGBA")

    if intensity == "heavy":
        num_far_flakes = int((width * height) * 0.0007)
        num_near_flakes = int((width * height) * 0.0004)
    else:
        num_far_flakes = int((width * height) * 0.0004)
        num_near_flakes = int((width * height) * 0.0002)

    far_layer = Image.new('RGBA', (width, height), (0,0,0,0))
    for _ in range(num_far_flakes):
        flake_size = random.randint(6,12)
        angle = random.randint(0,360)
        color = (255, 255, 255, random.randint(60,120))
        flake = create_snowflake(flake_size, color, angle)
        x = random.randint(0, width)
        y = random.randint(0, height)
        far_layer.alpha_composite(flake, (x,y))
    far_layer = far_layer.filter(ImageFilter.GaussianBlur(radius=2))

    near_layer = Image.new('RGBA', (width, height), (0,0,0,0))
    for _ in range(num_near_flakes):
        flake_size = random.randint(4,9)
        angle = random.randint(0,360)
        color = (255, 255, 255, random.randint(150,220))
        flake = create_snowflake(flake_size, color, angle)
        flake = flake.filter(ImageFilter.GaussianBlur(radius=0.5))
        x = random.randint(0, width)
        y = random.randint(0, height)
        near_layer.alpha_composite(flake, (x,y))

    snow_combined = Image.alpha_composite(img, far_layer)
    snow_combined = Image.alpha_composite(snow_combined, near_layer)
    return snow_combined

def apply_fog_effect(image, variant="light"):
    width, height = image.size

    fog_layer = Image.new('RGBA', (width, height), (0,0,0,0))
    draw = ImageDraw.Draw(fog_layer, 'RGBA')

    if variant == "light":
        veil_opacity = 50
        draw.rectangle((0,0,width,height), fill=(255,255,255,veil_opacity))
        fog_layer = fog_layer.filter(ImageFilter.GaussianBlur(radius=5))

    elif variant == "dense":
        veil_opacity = 100
        draw.rectangle((0,0,width,height), fill=(255,255,255,veil_opacity))
        for _ in range(10):
            x = random.randint(0, width)
            y = random.randint(0, height)
            r = random.randint(width//10, width//5)
            local_opacity = random.randint(80,150)
            draw.ellipse((x-r, y-r, x+r, y+r), fill=(255,255,255,local_opacity))
        fog_layer = fog_layer.filter(ImageFilter.GaussianBlur(radius=15))

    elif variant == "patchy":
        draw.rectangle((0,0,width,height), fill=(255,255,255,30))
        for _ in range(7):
            x = random.randint(0, width)
            y = random.randint(0, height)
            r = random.randint(width//8, width//4)
            local_opacity = random.randint(50,120)
            draw.ellipse((x-r, y-r, x+r, y+r), fill=(255,255,255,local_opacity))
        fog_layer = fog_layer.filter(ImageFilter.GaussianBlur(radius=20))

    foggy_image = Image.alpha_composite(image.convert("RGBA"), fog_layer)
    return foggy_image

def process_subset(subset):
    images_input = os.path.join(BASE_INPUT_DIR, subset, "images")
    labels_input = os.path.join(BASE_INPUT_DIR, subset, "labels")

    images_output = os.path.join(BASE_OUTPUT_DIR, subset, "images")
    labels_output = os.path.join(BASE_OUTPUT_DIR, subset, "labels")

    for filename in os.listdir(images_input):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            image_path = os.path.join(images_input, filename)
            try:
                image = Image.open(image_path).convert("RGBA")
            except Exception as e:
                print(f"Erreur lors de l'ouverture de {image_path}: {e}")
                continue
            base_name, ext = os.path.splitext(filename)

            # Charger le label correspondant
            label_filename = None
            for label_ext in ['.txt', '.json', '.xml']:  # Adapter selon le format de vos labels
                potential_label = os.path.join(labels_input, base_name + label_ext)
                if os.path.exists(potential_label):
                    label_filename = potential_label
                    break
            if label_filename is None:
                print(f"Aucun label trouvé pour {filename}. Skipping...")
                continue

            # Original
            output_og = os.path.join(images_output, f"{base_name}_og{ext}")
            save_image_with_conversion(image.copy(), output_og)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_og{os.path.splitext(label_filename)[1]}"))

            # Pluie (heavy)
            rain_heavy = apply_rain_effect(image.copy(), intensity="heavy")
            output_rain_heavy = os.path.join(images_output, f"{base_name}_rain_heavy{ext}")
            save_image_with_conversion(rain_heavy, output_rain_heavy)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_rain_heavy{os.path.splitext(label_filename)[1]}"))

            # Neige (heavy)
            snow_heavy = apply_snow_effect(image.copy(), intensity="heavy")
            output_snow_heavy = os.path.join(images_output, f"{base_name}_snow_heavy{ext}")
            save_image_with_conversion(snow_heavy, output_snow_heavy)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_snow_heavy{os.path.splitext(label_filename)[1]}"))

            # Brouillard (light)
            fog_light = apply_fog_effect(image.copy(), variant="light")
            output_fog_light = os.path.join(images_output, f"{base_name}_fog_light{ext}")
            save_image_with_conversion(fog_light, output_fog_light)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_fog_light{os.path.splitext(label_filename)[1]}"))

            # Brouillard (dense)
            fog_dense = apply_fog_effect(image.copy(), variant="dense")
            output_fog_dense = os.path.join(images_output, f"{base_name}_fog_dense{ext}")
            save_image_with_conversion(fog_dense, output_fog_dense)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_fog_dense{os.path.splitext(label_filename)[1]}"))

            # Brouillard (patchy)
            fog_patchy = apply_fog_effect(image.copy(), variant="patchy")
            output_fog_patchy = os.path.join(images_output, f"{base_name}_fog_patchy{ext}")
            save_image_with_conversion(fog_patchy, output_fog_patchy)
            shutil.copy(label_filename, os.path.join(labels_output, f"{base_name}_fog_patchy{os.path.splitext(label_filename)[1]}"))

            print(f"Traitement terminé pour {filename}")

# Appliquer le traitement à chaque sous-ensemble
for subset in SUBSETS:
    print(f"Traitement du sous-ensemble: {subset}")
    process_subset(subset)
    print(f"Terminé le sous-ensemble: {subset}\n")

print("Tous les traitements sont terminés.")


Traitement du sous-ensemble: train
Traitement terminé pour 00028_jpg.rf.5488adc8fdafafa82bd987be343a2b47.jpg
Traitement terminé pour images-36-_vflip_jpg.rf.f159996ff530b710bb010238518251c6.jpg
Traitement terminé pour 00064_jpg.rf.c973e907dbdcdacec7215cd8334a60f6.jpg
Traitement terminé pour Rat-Feature-1024x555-1-700x379_vflip_jpg.rf.36a2de467ebef18b2b2bb132d96f721f.jpg
Traitement terminé pour S_hflip_jpg.rf.92416d0e1b5aa19b67a419777b27a7d9.jpg
Traitement terminé pour I_hvflip_jpg.rf.4e09f65fd424901d8375a971f4f7ce8a.jpg
Traitement terminé pour C_vflip_jpg.rf.8376d8626afb1ed9cddb99c25b32e87b.jpg
Traitement terminé pour 00084_jpg.rf.efd41034b5f0c7ba14fa0b890e065438.jpg
Traitement terminé pour images-34-_hflip_jpg.rf.e41157fd0d5e36e1bd4c7d2590a7a15f.jpg
Traitement terminé pour images-9-_jpg.rf.ba3073b08225460d46b7a860b5ab631c.jpg
Traitement terminé pour 00094_jpg.rf.f3e6f66dc013ff96a19515458b7bf1b5.jpg
Traitement terminé pour F_vflip_jpg.rf.bb0d70b3ae77b78f7d0185c12eeb8cce.jpg
Traitement 

In [ ]:
!pip install ultralytics

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!curl -L "https://universe.roboflow.com/ds/8j88FD0WdD?key=K6uWl9ffUP" -o roboflow.zip
!unzip roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   3293      0 --:--:-- --:--:-- --:--:--  3287
100 86.4M  100 86.4M    0     0  45.4M      0  0:00:01  0:00:01 --:--:-- 72.8M
Archive:  roboflow.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
 extracting: test/images/00005_jpg.rf.a6f5c55a7f404c746ef07cf4d4d95464.jpg  
 extracting: test/images/00006_jpg.rf.9d87b58498416a826076c8aec44867ac.jpg  
 extracting: test/images/00008_jpg.rf.24fec05afb1fa5b81bc5bca0f232e32b.jpg  
 extracting: test/images/00017_jpg.rf.a71678a7b5834b899f40734c364f30fc.jpg  
 extracting: test/images/00020_jpg.rf.a7886e2b6a4d5c527ffb17624766ea25.jpg  
 extracting: test/images/00023-2-_jpg.rf.7e48d89266dc5881fb96a007a972f070.jpg  
 extracting: t

In [ ]:
!cat data.yaml


train: ../output/train/images
val: ../output/valid/images
test: ../output/test/images

nc: 2
names: ['butterfly', 'circular']

roboflow:
  workspace: yolo-mine
  project: minesweeper_final
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/yolo-mine/minesweeper_final/dataset/1

In [ ]:
!yolo detect train model=yolo11n.pt data=data2.yaml epochs=14 imgsz=640 device=0

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data2.yaml, epochs=14, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

In [ ]:
!yolo detect train model=yolo11n.pt data=data.yaml epochs=50 imgsz=640 device=0

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=N

In [ ]:
!yolo detect predict model=runs/detect/train2/weights/best.pt source=/content/mineneige2.webp

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

image 1/1 /content/mineneige2.webp: 512x640 1 butterfly, 67.9ms
Speed: 3.9ms preprocess, 67.9ms inference, 773.7ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo detect predict model=runs/detect/train4/weights/best.pt source=/content/IMG_6102.HEIC

Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

image 1/1 /content/IMG_6102.HEIC: 640x480 1 butterfly, 42.7ms
Speed: 3.5ms preprocess, 42.7ms inference, 563.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict16
💡 Learn more at https://docs.ultralytics.com/modes/predict
